# Data

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:

from backbone.utils.wfo_utils import run_strategy, run_wfo
from backbone.utils.general_purpose import calculate_units_size, diff_pips

import itertools
import numpy as np
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
from pandas import Timestamp
import numpy as np
import random
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta
from smartmoneyconcepts import smc

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

random.seed(42)

In [3]:
INITIAL_CASH = 10_000
MARGIN = 1/30

timezone = pytz.timezone("Etc/UTC")
date_from_get_data = datetime(2021, 10, 1, tzinfo=timezone)
date_to_get_data = datetime(2024, 9, 1, tzinfo=timezone)

#
limited_testing_start_date = Timestamp('2024-01-01 00:00:00', tz='UTC')
limited_testing_end_date = Timestamp('2024-10-01 00:00:00', tz='UTC')

In [4]:
groups = [
    'Forex',
    'Cash CFD',
    'Cash II CFD',
    'Metals CFD',
    'Crypto CFD',
    'Exotics',
    'Equities I CFD',
    'Equities II CFD',
    'Agriculture',
    'Commodities'
]

In [5]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

# symbols = mt5.symbols_get()

# tickers = [symbol.path.split('\\')[1] for symbol in symbols if (
#     ('Agriculture' in symbol.path)
#     or ('Cash CFD' in symbol.path)
#     or ('Cash II CFD' in symbol.path)
#     or ('Crypto CFD' in symbol.path)
#     or ('Equities I CFD' in symbol.path)
#     or ('Equities II CFD' in symbol.path)
#     or ('Agriculture' in symbol.path)
#     or ('Equities II CFD' in symbol.path)
#     or ('Commodities' in symbol.path)
#     or ('Forex' in symbol.path)
#     )
# ]

tickers = ['XMRUSD']

print(tickers)

intervals = [
    # mt5.TIMEFRAME_H8,
    # mt5.TIMEFRAME_H6,
    # mt5.TIMEFRAME_H4,
    # mt5.TIMEFRAME_H3,
    mt5.TIMEFRAME_H2,
    # mt5.TIMEFRAME_H1,
    # mt5.TIMEFRAME_M15,
    # mt5.TIMEFRAME_M10,
    # mt5.TIMEFRAME_M5,
]

symbols = get_data(tickers, intervals, date_from_get_data, date_to_get_data)

['XMRUSD']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
XMRUSD


# Comisiones

In [6]:
commissions = {}

for ticker in tickers:
    symbol_info = mt5.symbol_info_tick(ticker)
    
    avg_price = (symbol_info.bid + symbol_info.ask) / 2
    spread = symbol_info.ask - symbol_info.bid
    
    commissions[ticker] = round(spread / avg_price, 5)
    
commissions

{'XMRUSD': 0.01398}

In [7]:
max_start_date = None
intervals_start_dates = {}

tickers = np.unique(list(symbols.keys())).tolist()

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date


print(intervals_start_dates)
print(limited_testing_start_date)
print(limited_testing_end_date)

{16386: Timestamp('2021-10-01 00:00:00+0000', tz='UTC')}
2024-01-01 00:00:00+00:00
2024-10-01 00:00:00+00:00


In [ ]:
class SMC(Strategy):
    pip_value = None
    minimum_units = None
    maximum_units = None
    contract_volume = None
    opt_params = None
    risk=1
    
    risk_reward = 2
    pip_offset = 5
    
    window = 720
    
    unnmitigated_fvgs = pd.DataFrame(columns=['FVG', 'Top',	'Bottom', 'MitigatedIndex']) 
    unnmitigated_orderblocks = pd.DataFrame(columns=['OB', 'Top', 'Bottom', 'OBVolume', 'MitigatedIndex', 'Percentage']) 
    unswept_liquidity = pd.DataFrame(columns=['Liquidity', 'Level', 'End', 'Swept']) 


    def init(self):
        self.past_date = None
    
    def next(self):
        time = self.data.index[-1]
        
        # if time.date() == self.past_date:
        #     pass

        long_order = [order for order in self.orders if order.size > 0].pop()
        short_order = [order for order in self.orders if order.size < 0].pop()
        
        if not long_order and not self.position:
            self.past_date = time.date()
            print(self.past_date)
            price = self.data.Close[-1]
            
            positive_obs = self.unnmitigated_orderblocks[self.unnmitigated_orderblocks['OB'] == 1]
            positive_fvgs = self.unnmitigated_fvgs[self.unnmitigated_fvgs['FVG'] == 1]
            positive_liquidity = self.unswept_liquidity[self.unswept_liquidity['Liquidity'] == 1]
            
            
            if len(positive_obs) > 0 and (len(positive_liquidity) > 0):
                print('positive conditions')
                
                for _, row in positive_obs.iterrows():
                    
                    price = row.Top + (self.pip_offset * self.pip_value)
                    
                    for order in self.orders:
                        if order.limit == price:
                            return
                        
                    for trade in self.trades:
                        if trade.entry_price == price:
                            return
                    
                    sl_price = row.Bottom - (self.pip_offset * self.pip_value)
                    
                    sl_distance_in_pips = diff_pips(
                        price, 
                        sl_price, 
                        pip_value=self.pip_value
                    )
                    
                    tp_price = price + (self.risk_reward * sl_distance_in_pips * self.pip_value)
                    
                    units = calculate_units_size(
                        account_size=self.equity, 
                        risk_percentage=self.risk, 
                        stop_loss_pips=sl_distance_in_pips, 
                        pip_value=self.pip_value,
                        maximum_units=self.maximum_units,
                        minimum_units=self.minimum_units
                    )
                    
                    self.buy(
                        limit=price,
                        size=units,
                        sl=sl_price,
                        tp=tp_price
                    )
                    
        if not short_order and not self.position:     
            negative_obs = self.unnmitigated_orderblocks[self.unnmitigated_orderblocks['OB'] == -1]
            negative_fvgs = self.unnmitigated_fvgs[self.unnmitigated_fvgs['FVG'] == -1]
            negative_liquidity = self.unswept_liquidity[self.unswept_liquidity['Liquidity'] == -1]
            
            if len(negative_obs) > 0 and (len(negative_liquidity) > 0):
                print('negative conditions')
                for _, row in negative_obs.iterrows():
                    
                    price = row.Bottom - (self.pip_offset * self.pip_value)
                    
                    for order in self.orders:
                        if order.limit == price:
                            return
                        
                    for trade in self.trades:
                        if trade.entry_price == price:
                            return
                    
                    sl_price = row.Top + (self.pip_offset * self.pip_value)
                    
                    sl_distance_in_pips = diff_pips(
                        price, 
                        sl_price, 
                        pip_value=self.pip_value
                    )
                    
                    tp_price = price - (self.risk_reward * sl_distance_in_pips * self.pip_value)
                    
                    units = calculate_units_size(
                        account_size=self.equity, 
                        risk_percentage=self.risk, 
                        stop_loss_pips=sl_distance_in_pips, 
                        pip_value=self.pip_value,
                        maximum_units=self.maximum_units,
                        minimum_units=self.minimum_units
                    )
                    
                    self.sell(
                        limit=price,
                        size=units,
                        sl=sl_price,
                        tp=tp_price
                    )

            
            ohcl = pd.DataFrame(
                {
                    'Open': self.data.Open[-self.window:],
                    'High': self.data.High[-self.window:],
                    'Low': self.data.Low[-self.window:],
                    'Close': self.data.Close[-self.window:],
                    'Volume': self.data.Volume[-self.window:],
                }
            )
            
            # fvg = smc.fvg(ohcl, join_consecutive=False)
            # self.unnmitigated_fvgs = fvg[((fvg['FVG'] != 0) & (fvg['MitigatedIndex'] == 0))]
            
            swing_highs_lows = smc.swing_highs_lows(ohcl, swing_length = 10)
            orderblocks = smc.ob(ohcl, swing_highs_lows, close_mitigation = False)
            self.unnmitigated_orderblocks = orderblocks[(~orderblocks['OB'].isna()) & (orderblocks['MitigatedIndex'] == 0)]
            
            liq = smc.liquidity(ohcl, swing_highs_lows, range_percent = 0.01)
            self.unswept_liquidity = liq[(~liq['Liquidity'].isna()) & (liq['Swept'] == 0) ]
            
            
            
                
STRATEGY = SMC

# Analisis preliminar

In [11]:
strategies = [
    STRATEGY
]

experiments = list(itertools.product(
    tickers, intervals, strategies
))

stats_per_symbol = {}

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    # try:
    print(ticker, interval)
    
    commission = commissions[ticker]
    
    if ticker not in stats_per_symbol.keys():
        stats_per_symbol[ticker] = {}
    
    prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date]

    df_stats, stats = run_strategy(
        strategy=strategy,
        ticker=ticker,
        interval=interval,
        commission=0.0002, 
        prices=prices, 
        initial_cash=INITIAL_CASH, 
        margin=MARGIN,
        plot=True
    )

    performance = pd.concat([performance, df_stats])
    stats_per_symbol[ticker][interval] = stats

    # except Exception as e:
    #     print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)

XMRUSD 16386
2024-01-01
2024-01-02
2024-01-03
2024-01-04
2024-01-05
2024-01-06
2024-01-07
2024-01-08
2024-01-09
2024-01-10
2024-01-11
2024-01-12
2024-01-13
2024-01-14
2024-01-15
2024-01-16
2024-01-17
2024-01-18
2024-01-19
2024-01-20
negative conditions
2024-01-21
negative conditions
2024-01-22
negative conditions
2024-01-23
negative conditions
2024-01-24
negative conditions
2024-01-25
negative conditions
2024-01-26
negative conditions
2024-01-27
negative conditions
2024-01-28
negative conditions
2024-01-29
negative conditions
2024-01-30
negative conditions
2024-01-31
2024-02-01
2024-02-02
2024-02-03
2024-02-04
2024-02-05
negative conditions
2024-02-06
negative conditions
2024-02-07
negative conditions
2024-02-08
negative conditions
2024-02-09
negative conditions
2024-02-10
negative conditions
2024-02-11
negative conditions
2024-02-12
negative conditions
2024-02-13
negative conditions
2024-02-14
negative conditions
2024-02-15
negative conditions
2024-02-16
negative conditions
2024-02-17

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\backtesting.py:912: UserWarning: (2024-06-07 08:00:00+00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\backtesting.py:912: UserWarning: (2024-06-09 02:00:00+00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this tra

2024-06-09
2024-06-10
2024-06-11
positive conditions
(<Order size=2778.0, limit=1.2272, sl=1.1896, tp=1.3024, contingent=0>,)
(<Order size=2778.0, limit=1.2272, sl=1.1896, tp=1.3024, contingent=0>, <Order size=6695.0, limit=1.2351, sl=1.2195, tp=1.2663, contingent=0>)
(<Order size=2778.0, limit=1.2272, sl=1.1896, tp=1.3024, contingent=0>, <Order size=6695.0, limit=1.2351, sl=1.2195, tp=1.2663, contingent=0>, <Order size=4950.0, limit=1.2728, sl=1.2517, tp=1.315, contingent=0>)
(<Order size=2778.0, limit=1.2272, sl=1.1896, tp=1.3024, contingent=0>, <Order size=6695.0, limit=1.2351, sl=1.2195, tp=1.2663, contingent=0>, <Order size=4950.0, limit=1.2728, sl=1.2517, tp=1.315, contingent=0>, <Order size=2926.0, limit=1.3151, sl=1.2794, tp=1.3865, contingent=0>)
(<Order size=2778.0, limit=1.2272, sl=1.1896, tp=1.3024, contingent=0>, <Order size=6695.0, limit=1.2351, sl=1.2195, tp=1.2663, contingent=0>, <Order size=4950.0, limit=1.2728, sl=1.2517, tp=1.315, contingent=0>, <Order size=2926.0, l

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\backtesting.py:912: UserWarning: (2024-07-05 06:00:00+00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\backtesting.py:912: UserWarning: (2024-07-05 08:00:00+00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this tra

In [ ]:
stats._trades

In [ ]:
0/0

In [ ]:

rob_test = performance.groupby(['strategy', 'ticker']).agg({
    'return/dd': ['mean', 'std'],
    'stability_ratio': ['mean', 'std'],
    'trades': ['mean', 'std']
})

rob_test['return_dd_mean_std'] = rob_test[('return/dd', 'mean')] / rob_test[('return/dd', 'std')]


# rob_test = rob_test[(rob_test[('return/dd', 'mean')] >= 0) & (rob_test[('trades', 'mean')] > 10)].sort_values(by='return_dd_mean_std', ascending=False)

display(rob_test)

average_positive_tickers = rob_test.reset_index().ticker.tolist()

filter_performance = performance[performance['ticker'].isin(average_positive_tickers)]

portfolio = filter_performance.ticker.values.tolist()

intervals = filter_performance.interval.values.tolist()

filter_performance = filter_performance.sort_values(by=['ticker', 'interval'], ascending=[True, True])[
    [
        'strategy',
        'ticker',
        'interval',
        'stability_ratio',
        'trades',
        'return',
        'drawdown',
        'return/dd',
        'custom_metric',
        'win_rate',
        'avg_trade_percent',
        'Duration'
        ]
]#.drop_duplicates(subset=['ticker'], keep='first')

display(filter_performance.head(50))

In [ ]:
for ticker, interval in zip(portfolio, intervals):
    
    prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date]
    commission = commissions[ticker]

    df_stats = run_strategy(
        strategy=strategy,
        ticker=ticker,
        interval=interval,
        commission=commission, 
        prices=prices, 
        initial_cash=INITIAL_CASH, 
        margin=MARGIN,
        plot=True
    )
    # break
    

In [ ]:
0/0

# WFO

In [ ]:
lookback_bars_per_interval = {
    5: 3000,
    10: 2500,
    15: 2000,
    16385: 2000,
    16386: 1800,
    16387: 1800,
    16388: 1200,
    16390: 1200,
    16392: 1200,
}

In [ ]:
from backbone.utils.wfo_utils import optimization_function

validation_bars = 200
warmup_bars =  14

all_wfo_performances = pd.DataFrame()
all_opt_params = {}

params = {
    'adx_threshold' : list(np.arange(15, 26, 1)),
}

for index, row in filter_performance.iterrows():
    
    try:
        ticker = row.ticker
        interval = row.interval
        commission = commissions[ticker]
        print(ticker, interval)
        
        lookback_bars = lookback_bars_per_interval[interval]
        
        prices = symbols[ticker][interval]

        wfo_stats, df_stats, opt_params = run_wfo(
            strategy=STRATEGY,
            ticker=ticker,
            interval=interval,
            prices=prices,
            initial_cash=INITIAL_CASH,
            commission=commission,
            margin=MARGIN,
            optim_func=optimization_function,
            params=params,
            lookback_bars=lookback_bars,
            warmup_bars=warmup_bars,
            validation_bars=validation_bars,
            plot=False
        )
                
        if ticker not in all_opt_params.keys():
            all_opt_params[ticker] = {}

        all_opt_params[ticker][interval] = opt_params

        all_wfo_performances = pd.concat([all_wfo_performances, df_stats])
        
    
    except Exception as e:
        print(f'No se pudo ejecutar para el ticker {ticker}: {e}')
    
all_wfo_performances['return/dd'] = all_wfo_performances['return'] / -all_wfo_performances['drawdown']
all_wfo_performances['drawdown'] = -all_wfo_performances['drawdown']
all_wfo_performances['custom_metric'] = (all_wfo_performances['return'] / (1 + all_wfo_performances.drawdown)) * np.log(1 + all_wfo_performances.trades)

all_wfo_performances.drawdown_duration = pd.to_timedelta(all_wfo_performances.drawdown_duration)
all_wfo_performances.drawdown_duration = all_wfo_performances.drawdown_duration.dt.days

In [ ]:

performance = pd.DataFrame()

wfo_stats_per_symbol = {}

for index, row in filter_performance.iterrows():
    try:
        
        ticker = row.ticker
        interval = row.interval
        commission = commissions[ticker]
        print(ticker, interval)
        
        params = all_opt_params[ticker][interval]
        prices = symbols[ticker][interval].iloc[lookback_bars - warmup_bars + 1:]
        
        if ticker not in wfo_stats_per_symbol.keys():
            wfo_stats_per_symbol[ticker] = {}

        df_stats, wfo_stats = run_strategy(
            strategy=STRATEGY,
            ticker=ticker,
            interval=interval,
            commission=commission, 
            prices=prices, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN, 
            opt_params=params, 
            plot=True
        )

        wfo_stats_per_symbol[ticker][interval] = wfo_stats

        performance = pd.concat([performance, df_stats])

    except Exception as e:
        print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)

In [ ]:

rob_test = performance.groupby(['strategy', 'ticker']).agg({
    'return/dd': ['mean', 'std'],
    'stability_ratio': ['mean', 'std'],
    'trades': ['mean', 'std']
})

rob_test['return_dd_mean_std'] = rob_test[('return/dd', 'mean')] / rob_test[('return/dd', 'std')]

rob_test = rob_test[(rob_test[('return/dd', 'mean')] > 1) & (rob_test[('trades', 'mean')] > 10)].sort_values(by='return_dd_mean_std', ascending=False)

display(rob_test)

average_positive_tickers = rob_test.reset_index().ticker.tolist()

filter_performance = performance[performance['ticker'].isin(average_positive_tickers)]

portfolio = filter_performance.ticker.values.tolist()

intervals = filter_performance.interval.values.tolist()

filter_performance = filter_performance.sort_values(by=['ticker', 'interval'], ascending=[True, True])[
    [
        'strategy',
        'ticker',
        'interval',
        'stability_ratio',
        'trades',
        'return',
        'drawdown',
        'return/dd',
        'custom_metric',
        'win_rate',
        'avg_trade_percent',
        'Duration'
        ]
]#.drop_duplicates(subset=['ticker'], keep='first')

display(filter_performance.head(50))

In [ ]:
filtered_wfo_performance = performance[
    (
        ((performance['ticker'] == 'COCOA.c') & (performance['interval'] == 16385))
        | ((performance['ticker'] == 'ETHUSD') & (performance['interval'] == 16387))
    )
].sort_values(by='custom_metric', ascending=False)

filtered_wfo_performance

# Montecarlo

In [ ]:
# Crear una lista para almacenar los resultados de cada ticker
from backbone.utils.montecarlo_utils import montecarlo_statistics_simulation

data_drawdown = []
data_return = []
montecarlo_simulations = {}

all_drawdowns = pd.DataFrame()
all_returns = pd.DataFrame()

for index, row in filtered_wfo_performance.iterrows():
    ticker = row.ticker
    interval = row.interval
    try:
        
        print(f"Procesando ticker: {ticker} {interval}")
        trades_history = wfo_stats_per_symbol[ticker][interval]._trades
        eq_curve = wfo_stats_per_symbol[ticker][interval]._equity_curve
        
        # Simulación de Montecarlo para cada ticker (datos agregados)
        mc, synthetic_drawdown_curve, synthetic_return_curve = montecarlo_statistics_simulation(
            equity_curve=eq_curve,
            trade_history=trades_history, 
            n_simulations=100_000, 
            initial_equity=INITIAL_CASH, 
            threshold_ruin=0.9, 
            return_raw_curves=True,
            percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
        )
        
        montecarlo_simulations[f"{ticker}_{interval}"] = mc
        
        synthetic_drawdown_curve["ticker"] = f"{ticker}_{interval}"
        synthetic_return_curve["ticker"] = f"{ticker}_{interval}"
        
        all_drawdowns = pd.concat([all_drawdowns, synthetic_drawdown_curve])
        all_returns = pd.concat([all_returns, synthetic_return_curve])
    
    except Exception as e:
        print(f'hubo un problema con {ticker}_{interval}: {e}')

In [ ]:
dd_df = pd.DataFrame()
returns_df = pd.DataFrame()

for ticker, mc in montecarlo_simulations.items():
    mc = mc.rename(
        columns={
            'Drawdown (%)': f'drawdown_{ticker}',
            'Final Return (%)': f'return_{ticker}',
        }
    )

    if dd_df.empty:
        dd_df = mc[[f'drawdown_{ticker}']]
    
    else:
        dd_df = pd.merge(
            dd_df,
            mc[[f'drawdown_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
    if returns_df.empty:
        returns_df = mc[[f'return_{ticker}']]
    
    else:
        returns_df = pd.merge(
            returns_df,
            mc[[f'return_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
display(dd_df)
display(returns_df)


In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(12, 12))
sns.boxplot(data=all_drawdowns, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = all_drawdowns["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = all_drawdowns["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()


In [ ]:
# Generar el gráfico
plt.figure(figsize=(25, 18))
sns.boxplot(data=all_returns, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = all_returns["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = all_returns["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
